In [14]:
# file: export_q_roe_for_inspection.py

import pandas as pd
# 引入您项目中的真实模块
from load_data import load_financial_data_from_db
import config
import os

# ==============================================================================
#  1. 配置与加载
# ==============================================================================

# 使用我们已确认的单季度ROE列名
SINGLE_QUARTER_ROE_COLUMN = 'q_roe' 

print(f"--- 准备导出用于检查的 '{SINGLE_QUARTER_ROE_COLUMN}' 原始数据 ---")
df_raw = load_financial_data_from_db()

if df_raw is None or df_raw.empty:
    print("数据加载失败或数据为空，程序退出。")
else:
    print("数据加载成功！")
    
    # ==============================================================================
    #  2. 数据准备与清洗 (与计算脚本的前半部分一致)
    # ==============================================================================
    
    required_cols = ['ts_code', 'end_date', SINGLE_QUARTER_ROE_COLUMN]
    if not all(col in df_raw.columns for col in required_cols):
        print(f"[错误] 您的数据中缺少必要的列。需要 {required_cols}，但只找到了 {df_raw.columns.tolist()}")
    else:
        df_for_inspection = df_raw[required_cols].copy()
        
        # --- 标准化数据格式 ---
        df_for_inspection['end_date'] = pd.to_datetime(df_for_inspection['end_date'], errors='coerce')
        df_for_inspection[SINGLE_QUARTER_ROE_COLUMN] = pd.to_numeric(df_for_inspection[SINGLE_QUARTER_ROE_COLUMN], errors='coerce')
        
        # 为了检查，我们先不删除 q_roe 为空的行，而是将其保留为 NaN
        # 这样您就能在Excel里看到哪里是空的
        df_for_inspection.dropna(subset=['ts_code', 'end_date'], inplace=True)
        
        # 按公司和时间顺序排列，方便您在Excel里查看
        df_for_inspection.sort_values(by=['ts_code', 'end_date'], inplace=True)
        
        print(f"\n数据已排序，准备导出。")

        # ==============================================================================
        #  3. 核心步骤：导出到Excel
        # ==============================================================================
        
        print("\n--- 正在导出数据到Excel文件 ---")
        output_dir = config.BASE_OUTPUT_DIR
        os.makedirs(output_dir, exist_ok=True)
        
        # 定义一个清晰的文件名
        output_path = os.path.join(output_dir, "q_roe_source_data_for_inspection.xlsx")
        
        try:
            df_for_inspection.to_excel(output_path, index=False, engine='openpyxl')
            print(f"\n[成功] 数据已成功导出到: {output_path}")
            print("请打开此文件进行检查。")
        except Exception as e:
            print(f"\n[失败] 文件导出失败: {e}")

--- 准备导出用于检查的 'q_roe' 原始数据 ---
数据加载成功！

数据已排序，准备导出。

--- 正在导出数据到Excel文件 ---

[成功] 数据已成功导出到: /Users/alan-hopiy/Documents/个人研究/基本面量化/q_roe_source_data_for_inspection.xlsx
请打开此文件进行检查。


In [15]:
# file: export_q_roe_wide_format.py

import pandas as pd
from load_data import load_financial_data_from_db
import config
import os

# ==============================================================================
#  1. 配置与加载
# ==============================================================================

SINGLE_QUARTER_ROE_COLUMN = 'q_roe' 

print(f"--- 准备将 '{SINGLE_QUARTER_ROE_COLUMN}' 数据转换为宽表 ---")
df_raw = load_financial_data_from_db()

if df_raw is None or df_raw.empty:
    print("数据加载失败或数据为空，程序退出。")
else:
    print("数据加载成功！")
    
    # ==============================================================================
    #  2. 数据准备与清洗 (与上一版脚本完全一致)
    # ==============================================================================
    
    required_cols = ['ts_code', 'end_date', SINGLE_QUARTER_ROE_COLUMN]
    if not all(col in df_raw.columns for col in required_cols):
        print(f"[错误] 您的数据中缺少必要的列。需要 {required_cols}，但只找到了 {df_raw.columns.tolist()}")
    else:
        df = df_raw[required_cols].copy()
        
        # --- 标准化数据格式 ---
        df['end_date'] = pd.to_datetime(df['end_date'], errors='coerce')
        df[SINGLE_QUARTER_ROE_COLUMN] = pd.to_numeric(df[SINGLE_QUARTER_ROE_COLUMN], errors='coerce')
        df.dropna(subset=['ts_code', 'end_date', SINGLE_QUARTER_ROE_COLUMN], inplace=True)
        
        # --- 删除重复行 ---
        print(f"\n去重前的数据行数: {len(df)}")
        df.drop_duplicates(subset=['ts_code', 'end_date'], keep='last', inplace=True)
        print(f"去重后的数据行数: {len(df)}")
        
        # --- 排序 (虽然pivot_table不强制要求，但这是个好习惯) ---
        df.sort_values(by=['ts_code', 'end_date'], inplace=True)
        
        print("\n数据清洗与去重完成。")

        # ==============================================================================
        #  3. 核心步骤：将长表转换为宽表 (Pivot)
        # ==============================================================================
        
        print("\n正在执行 pivot_table 操作...")
        try:
            # index: 指定哪一列作为行索引
            # columns: 指定哪一列作为列标题
            # values: 指定用哪一列的数据填充单元格
            q_roe_panel = df.pivot_table(
                index='ts_code', 
                columns='end_date', 
                values=SINGLE_QUARTER_ROE_COLUMN
            )
            print("宽表创建成功！")
            
            # ==============================================================================
            #  4. 导出到Excel
            # ==============================================================================
            
            print("\n--- 正在导出宽表到Excel文件 ---")
            output_dir = config.BASE_OUTPUT_DIR
            os.makedirs(output_dir, exist_ok=True)
            
            output_path = os.path.join(output_dir, "q_roe_wide_format_for_inspection.xlsx")
            
            # index=True 是默认的，但这里写出来是为了清晰，表示我们需要把行索引(ts_code)也写入文件
            q_roe_panel.to_excel(output_path, index=True, engine='openpyxl')
            
            print(f"\n[成功] 已将宽表格式的数据导出到: {output_path}")
            print("请打开此文件进行检查。")

        except Exception as e:
            print(f"\n[失败] pivot_table或导出操作失败: {e}")

--- 准备将 'q_roe' 数据转换为宽表 ---
数据加载成功！

去重前的数据行数: 168624
去重后的数据行数: 101732

数据清洗与去重完成。

正在执行 pivot_table 操作...
宽表创建成功！

--- 正在导出宽表到Excel文件 ---

[成功] 已将宽表格式的数据导出到: /Users/alan-hopiy/Documents/个人研究/基本面量化/q_roe_wide_format_for_inspection.xlsx
请打开此文件进行检查。


In [16]:
# file: preprocess_q_roe_panel.py

import pandas as pd
from load_data import load_financial_data_from_db
import config
import os

# ==============================================================================
#  步骤 0: 加载并创建宽表面板数据 (逻辑同前)
# ==============================================================================
print("--- 步骤 0: 加载并创建宽表面板数据 ---")

df_raw = load_financial_data_from_db()
if df_raw is None or df_raw.empty:
    raise SystemExit("数据加载失败或数据为空，程序退出。")

df = df_raw[['ts_code', 'end_date', 'q_roe']].copy()
df['end_date'] = pd.to_datetime(df['end_date'], errors='coerce')
df['q_roe'] = pd.to_numeric(df['q_roe'], errors='coerce')
df.dropna(subset=['ts_code', 'end_date', 'q_roe'], inplace=True)
df.drop_duplicates(subset=['ts_code', 'end_date'], keep='last', inplace=True)
print("数据加载、清洗、去重完成。")

q_roe_panel = df.pivot_table(index='ts_code', columns='end_date', values='q_roe')
print(f"宽表创建成功，原始形状为: {q_roe_panel.shape}")


# ==============================================================================
#  步骤 1: 按数据量筛选公司 (剔除缺失值过多的行)
# ==============================================================================
print("\n--- 步骤 1: 按数据量筛选公司 ---")

MIN_DATA_POINTS = 12
valid_data_counts = q_roe_panel.count(axis=1)
panel_filtered = q_roe_panel[valid_data_counts >= MIN_DATA_POINTS]

print(f"设定的最少数据点: {MIN_DATA_POINTS}")
print(f"筛选前公司数量: {len(q_roe_panel)}")
print(f"筛选后公司数量: {len(panel_filtered)}")


# ==============================================================================
#  步骤 2: 处理极端值 (去极值/缩尾)
# ==============================================================================
print("\n--- 步骤 2: 处理极端值 (去极值) ---")
# (推荐使用5%和95%百分位法)
panel_winsorized = panel_filtered.copy()
print("采用5%和95%百分位法进行缩尾处理...")

for date_column in panel_winsorized.columns:
    series = panel_winsorized[date_column].dropna()
    q_low = series.quantile(0.05)
    q_high = series.quantile(0.95)
    panel_winsorized[date_column] = panel_winsorized[date_column].clip(lower=q_low, upper=q_high)

print("去极值处理完成。")


# ==============================================================================
#  步骤 3: 导出预处理完成的面板数据
# ==============================================================================
print("\n--- 步骤 3: 导出预处理完成的面板数据 ---")

# 现在的 panel_winsorized 就是您想要的、干净的q_roe数据面板
# 它已经经过了“行筛选”和“列缩尾”处理

output_dir = config.BASE_OUTPUT_DIR
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "q_roe_panel_processed.xlsx")
panel_winsorized.to_excel(output_path, index=True)

print(f"\n[成功] 已将预处理完成的 'q_roe' 面板数据导出到: {output_path}")
print("\n下一步，您应该基于这个干净的面板数据，来计算 ROE(TTM)。")

--- 步骤 0: 加载并创建宽表面板数据 ---
数据加载、清洗、去重完成。
宽表创建成功，原始形状为: (5412, 20)

--- 步骤 1: 按数据量筛选公司 ---
设定的最少数据点: 12
筛选前公司数量: 5412
筛选后公司数量: 5203

--- 步骤 2: 处理极端值 (去极值) ---
采用5%和95%百分位法进行缩尾处理...
去极值处理完成。

--- 步骤 3: 导出预处理完成的面板数据 ---

[成功] 已将预处理完成的 'q_roe' 面板数据导出到: /Users/alan-hopiy/Documents/个人研究/基本面量化/q_roe_panel_processed.xlsx

下一步，您应该基于这个干净的面板数据，来计算 ROE(TTM)。


In [17]:
# file: calculate_ttm_from_panel.py

import pandas as pd
import config
import os

# ==============================================================================
#  步骤 1: 加载预处理好的 q_roe 面板数据
# ==============================================================================
print("--- 步骤 1: 加载预处理好的 q_roe 面板数据 ---")

# 构建输入文件的完整路径
input_dir = config.BASE_OUTPUT_DIR
input_path = os.path.join(input_dir, "q_roe_panel_processed.xlsx")

try:
    # index_col=0 告诉pandas将Excel的第一列作为DataFrame的索引（也就是我们的ts_code）
    panel_processed = pd.read_excel(input_path, index_col=0)
    print(f"成功加载文件: {input_path}")
    print(f"加载的数据面板形状为: {panel_processed.shape}")
except FileNotFoundError:
    raise SystemExit(f"[错误] 输入文件未找到: {input_path}。请先运行上一步的 'preprocess_q_roe_panel.py' 脚本。")


# ==============================================================================
#  步骤 2: 核心计算 - 在宽表上逐行计算 TTM
# ==============================================================================
print("\n--- 步骤 2: 核心计算 - 逐行计算滚动的 ROE(TTM) ---")

# .apply(..., axis=1) 会将一个函数应用到DataFrame的每一行
# lambda row: ... 定义了一个匿名函数，它对每一行(row)执行滚动计算
# row.rolling(...) 对单行数据（一个公司的q_roe时间序列）进行滚动
# window=4 定义了4个季度的时间窗口
# min_periods=4 这是实现您需求的关键！它强制要求窗口内必须有4个非空数据点，否则结果就是NaN。
roe_ttm_panel = panel_processed.apply(
    lambda row: row.rolling(window=4, min_periods=4).mean(),
    axis=1
)

print("ROE(TTM) 计算完成。")
print(f"输出的TTM面板形状为: {roe_ttm_panel.shape} (应与输入形状一致)")


# ==============================================================================
#  步骤 3: 结果验证与导出
# ==============================================================================
print("\n--- 步骤 3: 结果验证与导出 ---")

# 验证一下结果，看看前几行的数据
print("\n--- TTM 面板数据预览 (前5行) ---")
print(roe_ttm_panel.head())

# --- 导出到Excel ---
output_path = os.path.join(input_dir, "roe_ttm_final_panel.xlsx")
roe_ttm_panel.to_excel(output_path, index=True)

print(f"\n[成功] 已将最终的 ROE(TTM) 面板数据导出到: {output_path}")

--- 步骤 1: 加载预处理好的 q_roe 面板数据 ---
成功加载文件: /Users/alan-hopiy/Documents/个人研究/基本面量化/q_roe_panel_processed.xlsx
加载的数据面板形状为: (5203, 20)

--- 步骤 2: 核心计算 - 逐行计算滚动的 ROE(TTM) ---
ROE(TTM) 计算完成。
输出的TTM面板形状为: (5203, 20) (应与输入形状一致)

--- 步骤 3: 结果验证与导出 ---

--- TTM 面板数据预览 (前5行) ---
           2020-03-31  2020-06-30  2020-09-30  2020-12-31  2021-03-31  \
ts_code                                                                 
000001.SZ         NaN         NaN         NaN    2.072450    2.117700   
000002.SZ         NaN         NaN         NaN    5.088350    5.066050   
000004.SZ         NaN         NaN         NaN    1.134275    1.139450   
000006.SZ         NaN         NaN         NaN    3.099375    3.526950   
000007.SZ         NaN         NaN         NaN   -5.530580   -5.132945   

           2021-06-30  2021-09-30  2021-12-31  2022-03-31  2022-06-30  \
ts_code                                                                 
000001.SZ    2.250075    2.390125    2.397550    2.511425    2.579050   
0

In [18]:
# file: calculate_final_z_scores.py

import pandas as pd
import config
import os

# ==============================================================================
#  步骤 1: 加载最终的 ROE(TTM) 面板数据
# ==============================================================================
print("--- 步骤 1: 加载最终的 ROE(TTM) 面板数据 ---")

# 构建输入文件的完整路径
input_dir = config.BASE_OUTPUT_DIR
input_path = os.path.join(input_dir, "roe_ttm_final_panel.xlsx")

try:
    # index_col=0 确保将第一列'ts_code'作为行索引
    roe_ttm_panel = pd.read_excel(input_path, index_col=0)
    print(f"成功加载文件: {input_path}")
    print(f"加载的TTM数据面板形状为: {roe_ttm_panel.shape}")
except FileNotFoundError:
    raise SystemExit(f"[错误] 输入文件未找到: {input_path}。请先运行上一步的 'calculate_ttm_from_panel.py' 脚本。")


# ==============================================================================
#  步骤 2: 计算每家公司的平均 ROE(TTM)
# ==============================================================================
print("\n--- 步骤 2: 计算每家公司的平均 ROE(TTM) ---")

# .mean(axis=1) 会对每一行（即每家公司）的所有TTM值求平均
# 它会自动忽略计算过程中的空值（NaN）
avg_roe_ttm = roe_ttm_panel.mean(axis=1)

print("平均值计算完成。")
print("\n--- 平均ROE(TTM)预览 (前5名) ---")
print(avg_roe_ttm.head())


# ==============================================================================
#  步骤 3: 对平均 ROE(TTM) 进行 Z-score 标准化
# ==============================================================================
print("\n--- 步骤 3: 进行Z-score标准化打分 ---")

# 计算全市场（我们样本内所有公司）的均值和标准差
market_mean = avg_roe_ttm.mean()
market_std = avg_roe_ttm.std()

print(f"全市场平均ROE(TTM): {market_mean:.4f}")
print(f"全市场标准差: {market_std:.4f}")

# 应用 Z-score 公式
z_scores = (avg_roe_ttm - market_mean) / market_std

print("Z-score 计算完成。")


# ==============================================================================
#  步骤 4: 整合并导出最终打分结果
# ==============================================================================
print("\n--- 步骤 4: 整合并导出最终打分结果 ---")

# 创建一个最终的打分DataFrame
final_scores_df = pd.DataFrame({
    'avg_roe_ttm': avg_roe_ttm,
    'z_score': z_scores
})

# 按Z-score降序排列，得分最高的排在最前面
final_scores_df.sort_values(by='z_score', ascending=False, inplace=True)

print("\n--- 最终Z-score打分排名预览 (前10名) ---")
print(final_scores_df.head(10))

# --- 导出到Excel ---
output_path = os.path.join(input_dir, "roe_factor_final_z_scores.xlsx")
final_scores_df.to_excel(output_path, index=True)

print(f"\n[成功] 已将最终的Z-score打分结果导出到: {output_path}")

--- 步骤 1: 加载最终的 ROE(TTM) 面板数据 ---
成功加载文件: /Users/alan-hopiy/Documents/个人研究/基本面量化/roe_ttm_final_panel.xlsx
加载的TTM数据面板形状为: (5203, 20)

--- 步骤 2: 计算每家公司的平均 ROE(TTM) ---
平均值计算完成。

--- 平均ROE(TTM)预览 (前5名) ---
ts_code
000001.SZ    2.492141
000002.SZ    1.884985
000004.SZ   -4.041324
000006.SZ    0.091944
000007.SZ    0.012852
dtype: float64

--- 步骤 3: 进行Z-score标准化打分 ---
全市场平均ROE(TTM): 1.5313
全市场标准差: 2.0810
Z-score 计算完成。

--- 步骤 4: 整合并导出最终打分结果 ---

--- 最终Z-score打分排名预览 (前10名) ---
           avg_roe_ttm   z_score
ts_code                         
300122.SZ     6.821726  2.542231
605117.SH     6.816724  2.539828
300979.SZ     6.651901  2.460626
600519.SH     6.627682  2.448988
605499.SH     6.542589  2.408098
834950.BJ     6.542142  2.407883
603195.SH     6.526389  2.400313
000568.SZ     6.516349  2.395489
603444.SH     6.415743  2.347144
300760.SZ     6.410048  2.344408

[成功] 已将最终的Z-score打分结果导出到: /Users/alan-hopiy/Documents/个人研究/基本面量化/roe_factor_final_z_scores.xlsx


In [2]:
# file: build_final_composite_growth_factor.py

import pandas as pd
import numpy as np
import config
import os

def calculate_growth_with_penalty(row, target_years):
    """
    【最终版】成长率计算函数。
    - 只有当期初和期末ROE(TTM)都为正时，才计算CAGR。
    - 否则，只要涉及负值或零，就给予-0.99的惩罚性低分。
    """
    series = row.dropna()
    target_period = target_years * 4
    if len(series) <= target_period: return np.nan
        
    latest_ttm = series.iloc[-1]
    base_ttm = np.nan
    
    lookback_periods = [target_period, target_period - 1, target_period + 1, target_period - 2]
    actual_period_in_quarters = np.nan
    for period in lookback_periods:
        if period > 0 and len(series) > period:
            potential_base = row.iloc[-(period + 1)]
            if pd.notna(potential_base):
                base_ttm = potential_base
                actual_period_in_quarters = period
                break
            
    if pd.isna(base_ttm):
        return np.nan
        
    if base_ttm > 0 and latest_ttm > 0:
        num_years = actual_period_in_quarters / 4.0
        return (latest_ttm / base_ttm) ** (1 / num_years) - 1
    else:
        return -10

# ==============================================================================
#  步骤 1: 加载预处理好的 ROE(TTM) 面板数据
# ==============================================================================
print("--- 步骤 1: 加载预处理好的 ROE(TTM) 面板数据 ---")
input_dir = config.BASE_OUTPUT_DIR
input_path = os.path.join(input_dir, "roe_ttm_final_panel.xlsx")
try:
    roe_ttm_panel = pd.read_excel(input_path, index_col=0)
    print(f"成功加载文件，数据面板形状为: {roe_ttm_panel.shape}")
except FileNotFoundError:
    raise SystemExit(f"[错误] 输入文件未找到: {input_path}。请先运行之前的脚本生成此文件。")

# ==============================================================================
#  步骤 2: 计算多周期成长率 (带惩罚项)
# ==============================================================================
print("\n--- 步骤 2: 计算带质量惩罚的多周期成长率 ---")
results_df = pd.DataFrame(index=roe_ttm_panel.index)
for years in [2, 3, 4]:
    print(f"正在计算 {years}年期 CAGR...")
    col_name = f'cagr_{years}yr_penalized'
    results_df[col_name] = roe_ttm_panel.apply(calculate_growth_with_penalty, axis=1, target_years=years)

# ==============================================================================
#  步骤 3: 独立对每个成长率因子进行Z-score打分
# ==============================================================================
print("\n--- 步骤 3: 独立进行Z-score打分 ---")
zscore_cols = []
for years in [2, 3, 4]:
    cagr_col = f'cagr_{years}yr_penalized'
    zscore_col = f'zscore_{years}yr_penalized'
    zscore_cols.append(zscore_col)
    
    mean = results_df[cagr_col].mean()
    std = results_df[cagr_col].std()
    
    results_df[zscore_col] = (results_df[cagr_col] - mean) / std
    print(f"完成 {zscore_col} 的计算。")
    
# ==============================================================================
#  步骤 4: 基于有效得分数量，筛选高质量公司
# ==============================================================================
print("\n--- 步骤 4: 基于有效得分数量，筛选高质量公司 ---")
MIN_VALID_SCORES = 2 
valid_score_counts = results_df[zscore_cols].count(axis=1)
results_df_filtered = results_df[valid_score_counts >= MIN_VALID_SCORES].copy()
print(f"筛选前公司数量: {len(results_df)}")
print(f"筛选后公司数量: {len(results_df_filtered)}")

# ==============================================================================
#  步骤 5: 对缺失的Z-score进行惩罚性填充
# ==============================================================================
print("\n--- 步骤 5: 对缺失的Z-score填充0值 (作为中性惩罚) ---")
results_df_filtered[zscore_cols] = results_df_filtered[zscore_cols].fillna(0)
print("缺失Z-score填充完成。")

# ==============================================================================
#  步骤 6: 合成得分，并对最终合成得分再次进行标准化
# ==============================================================================
print("\n--- 步骤 6: 合成得分并进行最终标准化 ---")
# 先计算临时的平均值
composite_avg = results_df_filtered[zscore_cols].mean(axis=1)

# 对这个平均值再做一次最终的Z-score标准化
final_mean = composite_avg.mean()
final_std = composite_avg.std()
results_df_filtered['composite_growth_score_final'] = (composite_avg - final_mean) / final_std
print("最终得分合成与标准化完成。")

# ==============================================================================
#  步骤 7: 排序并预览最终结果
# ==============================================================================
results_df_filtered.sort_values(by='composite_growth_score_final', ascending=False, inplace=True)
print("\n--- 最终打分排名预览 (前10名) ---")
# 为了更清晰地展示，我们只显示最终需要看的列
display_cols = [col for col in results_df_filtered.columns if 'cagr' not in col] # 不显示原始cagr值
print(results_df_filtered[display_cols].head(10))

# ==============================================================================
#  步骤 8: 导出最终结果
# ==============================================================================
output_path = os.path.join(input_dir, "ROE成长率Z值.xlsx")
results_df_filtered.to_excel(output_path, index=True)
print(f"\n[成功] 已将最终的、最完整的打分结果导出到: {output_path}")

--- 步骤 1: 加载预处理好的 ROE(TTM) 面板数据 ---
成功加载文件，数据面板形状为: (5203, 20)

--- 步骤 2: 计算带质量惩罚的多周期成长率 ---
正在计算 2年期 CAGR...
正在计算 3年期 CAGR...
正在计算 4年期 CAGR...

--- 步骤 3: 独立进行Z-score打分 ---
完成 zscore_2yr_penalized 的计算。
完成 zscore_3yr_penalized 的计算。
完成 zscore_4yr_penalized 的计算。

--- 步骤 4: 基于有效得分数量，筛选高质量公司 ---
筛选前公司数量: 5203
筛选后公司数量: 4790

--- 步骤 5: 对缺失的Z-score填充0值 (作为中性惩罚) ---
缺失Z-score填充完成。

--- 步骤 6: 合成得分并进行最终标准化 ---
最终得分合成与标准化完成。

--- 最终打分排名预览 (前10名) ---
           zscore_2yr_penalized  zscore_3yr_penalized  zscore_4yr_penalized  \
ts_code                                                                       
600711.SH              1.916357              0.725546              0.915574   
600150.SH              1.454233              1.025616              0.866119   
000581.SZ              1.900858              0.664733              0.659635   
601500.SH              1.369546              1.052747              0.752108   
002637.SZ              1.908251              0.598913              0.634460   
00262